In [32]:
###############################################################################################
#
#            The solution for Backend Intern Challenge - Summer 2018
# 
#       Coded by Ce Peng (Ben)    Address: 921 Greenbriar Ave, Ottawa, ON 
#
#               Cell: (613)-710-6281        Email:epengc@icloud.com
#
###############################################################################################

In [33]:
# Import the module of Python
import json,requests,math

In [34]:
#-------------------------Function definition-------------------------------------------
#  Function Name:      [api_data] = api_read(id_n)
#
#  Output: api_data    is the data online read by API
#  Input : id_n        id_n = 1 is the Basic Challenge data
#                      id_n = 2 is the Extra Challenge data
#
#  Usage:        Read data from API with address:
#           "https://backend-challenge-summer-2018.herokuapp.com/challenges.json?id=?&page=?"
#---------------------------------------------------------------------------------------
def api_read(id_n):
    address_prefix = "https://backend-challenge-summer-2018.herokuapp.com/challenges.json?id="+str(id_n)+"&page="
    api_response = requests.get(address_prefix+str(1))
    api_response.json()
    api_response_data = json.loads(api_response.text)
    # Determine how many pages data we have
    total_page = api_response_data['pagination']['total']
    per_page = api_response_data['pagination']['per_page']
    cur_page = api_response_data['pagination']['current_page']
    sum_page = math.ceil(total_page/per_page)
    # Read all data in one time
    # Initialize variables
    api_response = []
    api_menus_data = []
    api_response_data = {}
    # Read all menus data into api_menus_data
    for i in range(sum_page):
        address = address_prefix + str(i+1)
        api_response = requests.get(address)
        api_response.json()
        api_response_data = json.loads(api_response.text)
        for j in api_response_data['menus']:
            api_menus_data.append(j)
    return api_menus_data

In [35]:
#-------------------------Function definition-------------------------------------------
#  Function Name:   depth_tree(tree_node,trees,root_id)
#
#  Input: tree_node     the entries within api_menus_data
#         trees         api_menus_data
#         root_id       the id of a tree root
#
#  Usage: This is a recursive function which gets a pre-order travel of the tree 
#         with each root_id. The path of each travel is transfered by a global variable 
#         tree_path in form like:
#                        [[1,3,7,15,1],[2,4,5,6,8],...]
#---------------------------------------------------------------------------------------
def depth_tree(tree_node,trees,root_id):
    global tree_path
    tree_path.append(tree_node['id'])
    #print(tree_node['id'])
    if tree_node and not (root_id in tree_node['child_ids']):
        for i in tree_node['child_ids']:
            depth_tree(trees[i-1],trees,root_id)  

In [36]:
#-------------------------Function definition-------------------------------------------
#  Function Name:       node_check(node_idx,path,trees,drawback_idx)
#
#  Input: node_idx      The endpoint index of a path
#         path          One of a path within Paths
#         trees         The sum of the tree_node
#         drawback_idx  the drawback point index before the node_idx
#
#  Usage: This is a recursive function which checks the redundancy nodes on a path of a tree
#---------------------------------------------------------------------------------------
def node_check(node_idx,path,trees,drawback_idx):
    global checked_path
    global checked_node_idx
    checked_path = path
    if trees[path[drawback_idx]-1]['id']!=trees[path[node_idx]-1]['parent_id'] and node_idx > 1:
        del path[drawback_idx]
        drawback_idx = drawback_idx -1
        node_idx = node_idx -1
        checked_path = path
        checked_node_idx = checked_node_idx -1
        node_check(node_idx,path,trees,drawback_idx)
    if node_idx > 1:
        node_idx = node_idx -1
        drawback_idx = node_idx-1
        node_check(node_idx,path,trees,drawback_idx)

In [37]:
#------------------------------Main Procedure-----------------------------------#
api_menus_data = api_read(id_n=2) # id_n = 1 means besic challenge data; id_n = 2 means extra challenge data
print(api_menus_data) # To show the data which we have read
trees = api_menus_data # Change api_menus_data into a short name to remember easily

[{'id': 1, 'data': 'House', 'child_ids': [3, 4]}, {'id': 2, 'data': 'Company', 'child_ids': [8, 9, 11]}, {'id': 3, 'data': 'Kitchen', 'parent_id': 1, 'child_ids': [5, 18]}, {'id': 4, 'data': 'Living Room', 'parent_id': 1, 'child_ids': [6, 7, 19]}, {'id': 5, 'data': 'Sink', 'parent_id': 3, 'child_ids': []}, {'id': 6, 'data': 'TV', 'parent_id': 4, 'child_ids': []}, {'id': 7, 'data': 'Chair', 'parent_id': 4, 'child_ids': [20]}, {'id': 8, 'data': 'Meeting Rooms', 'parent_id': 2, 'child_ids': []}, {'id': 9, 'data': 'Kitchen', 'parent_id': 2, 'child_ids': [10]}, {'id': 10, 'data': 'Oven', 'parent_id': 9, 'child_ids': []}, {'id': 11, 'data': 'HR', 'parent_id': 2, 'child_ids': []}, {'id': 12, 'data': 'Computer', 'child_ids': [13, 14, 15]}, {'id': 13, 'data': 'CPU', 'parent_id': 12, 'child_ids': []}, {'id': 14, 'data': 'Motherboard', 'parent_id': 12, 'child_ids': []}, {'id': 15, 'data': 'Peripherals', 'parent_id': 12, 'child_ids': [16, 17, 21]}, {'id': 16, 'data': 'Mouse', 'parent_id': 15, 'chi

In [38]:
# Using a for loop to look for the root of the tree in trees
paths = [] # All paths of tree_path
for tree_node in trees:
    if not('parent_id' in tree_node):
        tree_path = []# Define a global variable to transfer the data of valid or invalid menus
        depth_tree(tree_node,trees,tree_node['id'])
        paths.append(tree_path)
print(paths) # To show how many paths we have from each root_id in preorder 

[[1, 3, 5, 18, 4, 6, 7, 20, 19], [2, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 21]]


In [39]:
# Read the paths and decode it into valid and invalid 
output = {"valid_menus":[],"invalid_menus":[]} # Initalize output variable
for path in paths:
    #print(path)
    path_cp = path
    i_path = 0
    trigger = 0
    menus_cell = {}
    while trigger == 0:
        if trees[path_cp[0]-1]['id'] in trees[path_cp[i_path]-1]['child_ids']: # Invaliad menus conditions
            checked_path = []
            checked_node_idx = i_path
            node_check(i_path,path_cp,trees,i_path-1)
            path_cp = checked_path
            i_path = checked_node_idx
            #print(path_cp)
            output['invalid_menus'].append({'root_id':trees[path_cp[0]-1]['id'],'children':path_cp[0:i_path+1]})
            del path_cp[i_path]
            i_path = i_path - 1
        if trees[path_cp[i_path]-1]['child_ids'] == []: # Valid menus conditions
            checked_path = []
            checked_node_idx = i_path
            node_check(i_path,path_cp,trees,i_path-1)
            path_cp = checked_path
            i_path = checked_node_idx
            #print('i_path=',i_path)
            #print('path_cp=',path_cp)
            output['valid_menus'].append({'root_id':trees[path_cp[0]-1]['id'],'children':path_cp[1:i_path+1]})
            del path_cp[i_path]
            i_path = i_path - 1
        i_path = i_path + 1
        if i_path >= len(path_cp):
            trigger = 1 

In [40]:
print(output)

{'valid_menus': [{'root_id': 1, 'children': [3, 5]}, {'root_id': 1, 'children': [3, 18]}, {'root_id': 1, 'children': [4, 6]}, {'root_id': 1, 'children': [4, 19]}, {'root_id': 2, 'children': [8]}, {'root_id': 2, 'children': [9, 10]}, {'root_id': 2, 'children': [11]}, {'root_id': 12, 'children': [13]}, {'root_id': 12, 'children': [14]}, {'root_id': 12, 'children': [15, 16]}, {'root_id': 12, 'children': [15, 17]}, {'root_id': 12, 'children': [15, 21]}], 'invalid_menus': [{'root_id': 1, 'children': [1, 4, 7, 20]}]}
